In [52]:
import pandas as pd
import numpy as np
import gzip
import json
import re
import os
import sys
import prince
import collections
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
%matplotlib inline

In [14]:
df = pd.read_csv('small.csv')

In [16]:
df

,tag1,tag2,tag3,tag4,tag5,isbn
0,historical-fiction,mystery,historical-mystery,medieval,historical,184737297X
1,short-stories,fiction,french,français,literature,0440083087
2,horror,stephen-king,fiction,sci-fi,science-fiction,0142427896
3,non-fiction,society,politics,history,economics,1400041694
4,fiction,christian-fiction,christian,giveaways,shelfari-medical,1463650132
5,mystery,england,mysteries,mood-comforting,milieu-small-town,0854563903
6,classics,literature,classic,american-literature,rory-gilmore-challenge,0743564421
7,econimics-and-finance,business,poleco,behaviour-science,risk-management,1118727010
8,fantasy,young-adult,ya,paranormal,series,0698143760
9,mystery,crime,fiction,mysteries,cornwall,0752844458


In [18]:
u_tags = pd.unique(df.iloc[:, 0:5].values.ravel('K'))

In [19]:
len(u_tags)

43

In [32]:
# add new columns from unique tags
def add_columns(df, columns):
    newcols = {}
    for i in range(len(columns)):
        newcols['%s' % columns[i]] = [0]*10
    return pd.concat([df, pd.DataFrame(newcols, index=df.index)], axis=1)

In [36]:
test = u_tags

dftags = add_columns(df, test)
dftags.drop(df.iloc[:, 0:5], axis = 1, inplace = True) 

In [80]:
df

,tag1,tag2,tag3,tag4,tag5,isbn
0,historical-fiction,mystery,historical-mystery,medieval,historical,184737297X
1,short-stories,fiction,french,français,literature,0440083087
2,horror,stephen-king,fiction,sci-fi,science-fiction,0142427896
3,non-fiction,society,politics,history,economics,1400041694
4,fiction,christian-fiction,christian,giveaways,shelfari-medical,1463650132
5,mystery,england,mysteries,mood-comforting,milieu-small-town,0854563903
6,classics,literature,classic,american-literature,rory-gilmore-challenge,0743564421
7,econimics-and-finance,business,poleco,behaviour-science,risk-management,1118727010
8,fantasy,young-adult,ya,paranormal,series,0698143760
9,mystery,crime,fiction,mysteries,cornwall,0752844458


In [86]:
#set([x for x in  if l.count(x) > 1])
alltags = [item for elm in df.iloc[:, 0:5].values for item in elm]
dup_tags = set([x for x in alltags if alltags.count(x) > 1])

In [93]:
def get_tag_counts_per_book(df, dup_tags):
    countdict ={}
    for book in df['isbn']:
        row = (df[df['isbn'] == book].iloc[:,:5].values)[0]
        counts = [int(val in dup_tags) for val in row]
        countdict[book] = {i: j for i,j in zip(row, counts)}
    return countdict

In [95]:
test_count = get_tag_counts_per_book(df, dup_tags)
print(test_count)
print(dup_tags)

{'184737297X': {'historical-fiction': 0, 'mystery': 1, 'historical-mystery': 0, 'medieval': 0, 'historical': 0}, '0440083087': {'short-stories': 0, 'fiction': 1, 'french': 0, 'français': 0, 'literature': 1}, '0142427896': {'horror': 0, 'stephen-king': 0, 'fiction': 1, 'sci-fi': 0, 'science-fiction': 0}, '1400041694': {'non-fiction': 0, 'society': 0, 'politics': 0, 'history': 0, 'economics': 0}, '1463650132': {'fiction': 1, 'christian-fiction': 0, 'christian': 0, 'giveaways': 0, 'shelfari-medical': 0}, '0854563903': {'mystery': 1, 'england': 0, 'mysteries': 1, 'mood-comforting': 0, 'milieu-small-town': 0}, '0743564421': {'classics': 0, 'literature': 1, 'classic': 0, 'american-literature': 0, 'rory-gilmore-challenge': 0}, '1118727010': {'econimics-and-finance': 0, 'business': 0, 'poleco': 0, 'behaviour-science': 0, 'risk-management': 0}, '0698143760': {'fantasy': 0, 'young-adult': 0, 'ya': 0, 'paranormal': 0, 'series': 0}, '0752844458': {'mystery': 1, 'crime': 0, 'fiction': 1, 'mysteries

In [96]:
#get_tag_counts_per_book(df, u_tags)
#alltags = [item for elm in df.iloc[:, 0:5].values for item in elm]
#collections.Counter(alltags)